In [ ]:
import feedparser
import re, os
import requests
import threading, tqdm
from PIL import Image
from io import BytesIO

In [ ]:
rss_url = "https://www.pixivision.net/zh-tw/rss"
feed = feedparser.parse(rss_url) 

# 檢查有沒有成功載入
if feed.bozo == 0:
    print("伊莉亞成功載入 RSS 了呢～♡")
else:
    print("哎呀，好像有點問題…請檢查 URL 喔～")

In [ ]:
def imageDL(url, savefilename):
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed. code: {response.status_code}, url: {url}")
        return

    img = Image.open(BytesIO(response.content))
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    img.save(savefilename)

In [ ]:
_threads = []
saved = []
[('', ''), ]
for entry in feed.entries[10:20]:
    # print(entry.title)      # 標題
    # print(entry.link)       # 原始網址
    # print(entry.summary)    # 摘要 HTML
    # print(entry.href)       # 連結網址
    img_link = re.sub(r'i.pximg.net/c/128x128_80', r'i.pixiv.cat', entry.href) # 替換網址
    img_date = re.search(r'img/(\d+)/(\d+)/(\d+)/', img_link)
    img_date = f"{img_date.group(1)}_{img_date.group(2)}{img_date.group(3)}" # 取得日期
    img_id = re.search(r'/(\d+)_', img_link).group(1) # 取得圖片 ID
    filename = f'imgs/img_{img_date}_{img_id}.jpg'
    saved.append(filename)
    _threads.append(threading.Thread(target=imageDL, args=(img_link, filename), name=filename))
    # display(img_link)

for t in _threads:
    t.start()
for t in _threads:
    t.join()

# for t in saved:
#     img = Image.open(t)
#     display(img)